In [1]:
import pandas as pd
import json

In [2]:
json_as_list = []

with open('./new_york_times_bestsellers-raw.json', 'r') as f:
    for line in f.readlines():
        json_as_list.append(json.loads(line))

In [3]:
normalized = pd.json_normalize(json_as_list)

df = pd.DataFrame.from_dict(normalized)

df.head(5)

,amazon_product_url,author,description,publisher,title,_id.$oid,bestsellers_date.$date.$numberLong,published_date.$date.$numberLong,price.$numberInt,rank.$numberInt,rank_last_week.$numberInt,weeks_on_list.$numberInt,price.$numberDouble
0,http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"Odd Thomas, who can communicate with the dead,...",Bantam,ODD HOURS,5b4aa4ead3089013507db18b,1211587200000,1212883200000,27,1,0,1,NaN
1,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,"Little, Brown",THE HOST,5b4aa4ead3089013507db18c,1211587200000,1212883200000,NaN,2,1,3,25.99
2,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,St. Martin's,LOVE THE ONE YOU'RE WITH,5b4aa4ead3089013507db18d,1211587200000,1212883200000,NaN,3,2,2,24.95
3,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,A Massachusetts state investigator and his tea...,Putnam,THE FRONT,5b4aa4ead3089013507db18e,1211587200000,1212883200000,NaN,4,0,1,22.95
4,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,Doubleday,SNUFF,5b4aa4ead3089013507db18f,1211587200000,1212883200000,NaN,5,0,1,24.95


In [4]:
df.to_csv('new_york_times_bestsellers.csv')

In [4]:
df_dropna = df.drop(columns=['price.$numberInt']).dropna(axis=0, how='any', inplace=False).reset_index(drop=True)

In [5]:
df_dropna.columns

Index(['amazon_product_url', 'author', 'description', 'publisher', 'title',
       '_id.$oid', 'bestsellers_date.$date.$numberLong',
       'published_date.$date.$numberLong', 'rank.$numberInt',
       'rank_last_week.$numberInt', 'weeks_on_list.$numberInt',
       'price.$numberDouble'],
      dtype='object')

In [6]:
df_renamed = df_dropna.rename(columns={
    "_id.$oid": "oid",
    "bestsellers_date.$date.$numberLong": "bestsellers_date.numberLong",
    "published_date.$date.$numberLong": "published_date.numberLong",
    "rank.$numberInt": "rank.numberInt",
    "rank_last_week.$numberInt": "rank_last_week.numberInt",
    "weeks_on_list.$numberInt": "weeks_on_list.numberInt",
    "price.$numberDouble": "price.numberDouble"
}, inplace=False)

df_renamed.head(5)

,amazon_product_url,author,description,publisher,title,oid,bestsellers_date.numberLong,published_date.numberLong,rank.numberInt,rank_last_week.numberInt,weeks_on_list.numberInt,price.numberDouble
0,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,"Little, Brown",THE HOST,5b4aa4ead3089013507db18c,1211587200000,1212883200000,2,1,3,25.99
1,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,St. Martin's,LOVE THE ONE YOU'RE WITH,5b4aa4ead3089013507db18d,1211587200000,1212883200000,3,2,2,24.95
2,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,A Massachusetts state investigator and his tea...,Putnam,THE FRONT,5b4aa4ead3089013507db18e,1211587200000,1212883200000,4,0,1,22.95
3,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,Doubleday,SNUFF,5b4aa4ead3089013507db18f,1211587200000,1212883200000,5,0,1,24.95
4,http://www.amazon.com/Sundays-at-Tiffanys-Jame...,James Patterson and Gabrielle Charbonnet,A woman finds an unexpected love,"Little, Brown",SUNDAYS AT TIFFANY’S,5b4aa4ead3089013507db190,1211587200000,1212883200000,6,3,4,24.99


In [7]:
casting_dict = {
    "bestsellers_date.numberLong": int,
    "published_date.numberLong": int,
    "rank.numberInt": int,
    "rank_last_week.numberInt": int,
    "weeks_on_list.numberInt": int,
    "price.numberDouble": float
}

df_casted = df_renamed.astype(dtype=casting_dict, errors='ignore')

df_casted.dtypes

amazon_product_url              object
author                          object
description                     object
publisher                       object
title                           object
oid                             object
bestsellers_date.numberLong      int64
published_date.numberLong        int64
rank.numberInt                   int64
rank_last_week.numberInt         int64
weeks_on_list.numberInt          int64
price.numberDouble             float64
dtype: object

In [8]:
df_casted.to_json('new_york_times_bestsellers.json')

In [9]:
with open('new_york_times_bestsellers.json') as f:
    data = json.load(f)
    
df_2 = pd.DataFrame.from_dict(data)

df_2.dtypes

amazon_product_url              object
author                          object
description                     object
publisher                       object
title                           object
oid                             object
bestsellers_date.numberLong      int64
published_date.numberLong        int64
rank.numberInt                   int64
rank_last_week.numberInt         int64
weeks_on_list.numberInt          int64
price.numberDouble             float64
dtype: object